# Calculation

**Table of contents**
- Overview
- Setup
  - Authentication Token
- Query
  - Output Description
- Related Links

## Overview

The Calculation endpoint provides a versatile, general-purpose solution for emissions calculations across any activity type supported by the system. Unlike scope-specific endpoints that are tailored for particular use cases, this endpoint offers flexibility to handle a wide range of scenarios where specialized endpoints might not exist. It allows users to:

- Calculate emissions from various activities like energy use, transportation, and material consumption
- Specify location data to get region-specific calculations
- Include date information to ensure time-relevant emission factors are used
- Receive detailed breakdown of emissions by gas type (CO2, CH4, N2O, etc.)

The Calculation API serves as a versatile, general-purpose solution for emissions calculations that aren't covered by specialized endpoints. It provides flexibility to handle a wide range of activity types using a consistent interface, automatically selecting appropriate emission factors based on the provided parameters. This API is particularly valuable when dealing with uncommon activities, when specialized endpoints don't exist for specific use cases, or when organizations need to quickly implement emissions calculations without developing multiple integrations.

## Setup


Ensure that you have Python installed in your system. Python 3+ is required.

<b>Note:</b> To run this notebook, you must first add your credentials to `config.read('../../../auth/secrets.ini')` in the following format:


```
[EAPI]
api.api_key = <Your GHG APIs API key>
api.client_id = <Your GHG APIs client Id>
api.org_id = <Your GHG APIs Org Id>

```

In [ ]:
# Install the packages below using pip/pip3 based on your python version.
%pip install pandas configparser IPython requests

In [1]:
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

### Authentication Token



Run the following code snippet to generate the Auth Bearer Token by using your api_key, client_id and org_id configured in secrets.ini.

In [3]:
config = configparser.RawConfigParser()
config.read(['../../../auth/secrets.ini','../../../auth/config.ini'])

EAPI_API_KEY        = config.get('EAPI', 'api.api_key')
EAPI_TENANT_ID      = config.get('EAPI', 'api.tenant_id')
EAPI_ORG_ID         = config.get('EAPI', 'api.org_id') 

EAPI_AUTH_ENDPOINT  = config.get('EAPI', 'api.auth_endpoint')
EAPI_BASE_URL       = config.get('EAPI', 'api.base_url')
EAPI_ENDPOINT       = f"{EAPI_BASE_URL}/calculation"

EAPI_AUTH_CLIENT_ID = 'saascore-' + EAPI_TENANT_ID
EAPI_CLIENT_ID      = 'ghgemissions-' + EAPI_TENANT_ID

auth_request_headers: dict = {}
auth_request_headers["X-IBM-CLIENT-ID"] = EAPI_AUTH_CLIENT_ID
auth_request_headers["X-API-KEY"] = EAPI_API_KEY

verify = True

auth_url = f"{EAPI_AUTH_ENDPOINT}?orgId={EAPI_ORG_ID}"
              
response = requests.get(url = auth_url,
                        headers = auth_request_headers,
                        verify  = verify
                       )
if response.status_code == 200:
    jwt_token = response.text
    print("Authentication Success")
else:     
    print("Authentication Failed")
    print(response.text)

Authentication Success


## Query

The example request payload (below) queries IBM Envizi - Emissions API with location data (India), activity information (adhesives with a value of 12 USD), and a specific date (January 4, 2024).

In [4]:
payload = {
  "location": {
    "country": "ind"
  },
  "activity": {
    "value": [
      12
    ],
    "type": "Adhesives",
    "unit": [
      "USD"
    ]
  },
  "time": {
    "date": "2024-01-04"
  }
}

In [5]:
# Create the query headers
request_headers: dict = {}
request_headers["Content-Type"] = "application/json"
request_headers["x-ibm-client-id"] = EAPI_CLIENT_ID
request_headers["Authorization"] = "Bearer " + jwt_token

# Submit the request
response = requests.post(EAPI_ENDPOINT, 
                         headers = request_headers, 
                         data = json.dumps(payload))

For more information about allowable parameters for the payload, please see [Emissions API Developer Guide]().

In [6]:
if response.text != "":
    # Get the response as json
    response_json = response.json()
    
    # Get json and convert to dataframe
    json_str = json.dumps(response_json)
    dict = json.loads(json_str)
    dataframe = pd.json_normalize(dict) 
    
    # display
    print("\n\n")
    pd.set_option('display.max_colwidth', None)
    display( dataframe) 
else:
    print("Empty Response")

,transactionId,totalCO2e,CO2,CH4,indirectCO2e,unit,description
0,9eb24f2a-7ef1-49cc-a057-0f31b6f36e16,9.096,8.376,0.6,0.396,kgCO2e,The Adhesives emissions factor used to calculate this result was obtained from the Managed - eGRID & US Climate Leaders factor set for the area Earth year 2023.


### Output Description

<b>transactionId</b> - An Emissions API transaction id.

<b>totalCO2e</b>     - The total emissions as CO<sub>2</sub> equivalent (CO<sub>2</sub>e)

<b>CO2</b>           - The amount of CO<sub>2</sub> (Carbon Dioxide) in the CO<sub>2</sub>e value.

<b>CH4</b>           - The amount of CH<sub>4</sub> (Methane) in the CO<sub>2</sub>e value.

<b>N2O</b>           - The amount of N<sub>2</sub>O (Nitrous Oxide) in the CO<sub>2</sub>e value.

<b>HFC</b>           - The amount of HFCs (Hydrofluorocarbons) in the CO<sub>2</sub>e value.

<b>PFC</b>           - The amount of PFCs (Perfluorocarbons) in the CO<sub>2</sub>e value.

<b>SF6</b>           - The amount of SF<sub>6</sub> (Sulphur Hexafluoride) in the CO<sub>2</sub>e value.

<b>NF3</b>           - The amount of NF<sub>3</sub> (Nitrogen Trifluoride) in the CO<sub>2</sub>e value.

<b>bioCo2</b>        - The amount of bio CO<sub>2</sub> in the CO<sub>2</sub> value.

<b>indirectCo2e</b>  - The amount of CO<sub>2</sub>e that is indirect in the CO<sub>2</sub>e value.

<b>unit</b>          - The unit of measure of the values.

<b>description</b>   - A description of the source factor set of the factor used in the calculation.

## Related Links

[GHG Emissions API Developer Guide](https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/GHG+Emissions+API)